In [1]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
import matplotlib.pyplot as plt
import seaborn as sns ; sns.set()


load_dotenv('BDD_URL.env')
BDD_URL = os.environ['BDD_URL']
engine = create_engine(BDD_URL)



In [ ]:
SQL = "SELECT * from principal.title_ratings "
basics_df = pd.read_sql(SQL, engine)

engine.dispose()



In [4]:
basics_df[basics_df["averageRating"]>9]

,tconst,averageRating,numVotes
1147,tt0002582,9.1,18
2375,tt0008528,9.3,15
12047,tt0027944,9.4,7
13293,tt0029523,9.5,10
18577,tt0035924,9.1,9
...,...,...,...
1398111,tt9915084,9.2,27
1398133,tt9915898,9.4,3248
1398159,tt9916400,9.4,6
1398164,tt9916460,9.4,18


In [2]:
SQL = "SELECT * from principal.title_basics limit 10000"
df = pd.read_sql(SQL, engine)

In [3]:
df.titleType.value_counts()

titleType
movie          7059
tvSeries       1171
tvMovie         898
video           425
tvMiniSerie     296
tvEpisode       114
videoGame        32
tvSpecial         5
Name: count, dtype: int64

In [18]:
listclass=['decades1','decades2','decades3','decades4','decades5', 'decades6','decades7','decades8','decades9','decades10','decades11','decades12','decades13','decades14']
listvalues=[1900,1910,1920,1930,1940,1950,1960,1970,1980,1990,2000,2010,2020,2030,2040]
dff=pd.cut(df['startYear'], bins=listvalues, labels=listclass)
dff.value_counts()

startYear
decades10    2820
decades9     1522
decades3     1011
decades8      922
decades4      668
decades6      635
decades7      628
decades12     525
decades2      504
decades5      455
decades11     143
decades1       13
decades13       1
decades14       0
Name: count, dtype: int64

In [40]:
SQL = '''SELECT "averageRating", "startYear", "runtimeMinutes" from principal.title_basics join principal.title_ratings on principal.title_basics."tconst" = principal.title_ratings."tconst"  limit 50000;'''
df = pd.read_sql(SQL, engine)
df



,averageRating,startYear,runtimeMinutes
0,7.7,1974,60.0
1,7.5,1973,60.0
2,6.5,1999,138.0
3,6.6,1984,50.0
4,8.4,1975,1800.0
...,...,...,...
49995,7.1,2003,10.0
49996,4.8,2005,95.0
49997,3.8,2004,139.0
49998,4.7,1960,86.0


In [47]:
df["averageRating"].fillna(df["averageRating"].mean(), inplace=True)
df["startYear"].fillna(df["startYear"].mean(), inplace=True)
df["runtimeMinutes"].fillna(df["runtimeMinutes"].mean(), inplace=True)
df.describe()

,averageRating,startYear,runtimeMinutes
count,50000.000000,50000.000000,50000.000000
mean,6.338264,1984.843180,75.344908
std,1.345237,22.038704,55.699979
min,1.000000,1888.000000,1.000000
25%,5.500000,1975.000000,37.000000
50%,6.500000,1994.000000,84.000000
75%,7.300000,2001.000000,97.000000
max,10.000000,2023.000000,5220.000000


In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   averageRating   50000 non-null  float64
 1   startYear       50000 non-null  int64  
 2   runtimeMinutes  50000 non-null  float64
dtypes: float64(2), int64(1)
memory usage: 1.1 MB


In [49]:


y=df["averageRating"]
X=df[["startYear","runtimeMinutes"]]


In [51]:
# Import des librairies
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_error
import numpy as np

# Création de la partition train / test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,
random_state=42)


scaler = RobustScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


# Déclaration du modèle
model = LinearRegression()
# Entrainement du modèle
model.fit(X_train, y_train)

# Prédiction sur nouvelles valeurs
y_pred = model.predict(X_test)

print('score =',model.score(X_train, y_train))

# Accès au score 
print('R2 =',r2_score(y_test, y_pred))

mse = mean_squared_error(y_test, y_pred)
print("MSE = ", mse)

rmse = np.sqrt(mse)
print("RMSE =", rmse)

# Calculer l'erreur absolue moyenne (MAE)
mae = mean_absolute_error(y_test, y_pred)
print("MAE =", mae)

score = 0.008184652874245746
R2 = 0.009417571581802453
MSE =  1.7835061813271615
RMSE = 1.3354797569889112
MAE = 1.0481552974750274
